In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly 

In [3]:
# Download Life expectancy based on economic freedom with time line

'''
Life expectancy:
https://www.kaggle.com/kumarajarshi/life-expectancy-who

Economic freedom 
https://www.kaggle.com/gsutters/economic-freedom 

Country to continent
https://www.kaggle.com/andradaolteanu/country-mapping-iso-continent-region 
'''

'\nLife expectancy:\nhttps://www.kaggle.com/kumarajarshi/life-expectancy-who\n\nEconomic freedom \nhttps://www.kaggle.com/gsutters/economic-freedom \n\nCountry to continent\nhttps://www.kaggle.com/andradaolteanu/country-mapping-iso-continent-region \n'

In [4]:
# File location 

economic_freedom = 'drive/My Drive/NYU/Programming_for_DS/Data/efw_cc.csv'
life_expectancy  = 'drive/My Drive/NYU/Programming_for_DS/Data/Life Expectancy Data.csv'
continents       = 'drive/My Drive/NYU/Programming_for_DS/Data/continents2.csv'

In [5]:
df_ef = pd.read_csv(economic_freedom)
df_le = pd.read_csv(life_expectancy)
conti = pd.read_csv(continents)

In [6]:
#Common years between two datasets

ef_date = df_ef.year.unique()
df_date = df_le.Year.unique()
common_years = np.intersect1d(ef_date, df_date)

print(common_years)

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015]


In [7]:
# Economic Freedom

df_ef.head()

,year,ISO_code,countries,ECONOMIC FREEDOM,rank,quartile,1a_government_consumption,1b_transfers,1c_gov_enterprises,1d_top_marg_tax_rate,1_size_government,2a_judicial_independence,2b_impartial_courts,2c_protection_property_rights,2d_military_interference,2e_integrity_legal_system,2f_legal_enforcement_contracts,2g_restrictions_sale_real_property,2h_reliability_police,2i_business_costs_crime,2j_gender_adjustment,2_property_rights,3a_money_growth,3b_std_inflation,3c_inflation,3d_freedom_own_foreign_currency,3_sound_money,4a_tariffs,4b_regulatory_trade_barriers,4c_black_market,4d_control_movement_capital_ppl,4_trade,5a_credit_market_reg,5b_labor_market_reg,5c_business_reg,5_regulation
0,2016,ALB,Albania,7.54,34.0,1.0,8.232353,7.509902,8.0,8.0,7.935564,2.668222,3.145462,4.512228,8.333333,4.166667,4.387444,6.485287,6.933500,6.215401,0.948718,5.071814,8.986454,9.484575,9.74360,10.0,9.553657,8.963556,7.489905,10.00000,6.406138,8.214900,7.098562,6.916278,6.705863,6.906901
1,2016,DZA,Algeria,4.99,159.0,4.0,2.150000,7.817129,0.0,4.5,3.616782,4.186704,4.327113,4.689952,4.166667,5.000000,4.507538,6.626692,6.136845,6.737383,0.820513,4.690743,6.955962,8.339152,8.72046,5.0,7.253894,6.872533,2.481294,5.56391,1.590362,4.127025,5.100509,5.029513,5.676956,5.268992
2,2016,AGO,Angola,5.17,155.0,4.0,7.600000,8.886739,0.0,9.5,6.496685,1.843129,1.974566,2.512364,3.333333,4.166667,2.302200,5.455882,3.016104,4.291197,0.846154,2.963635,9.385679,4.986742,3.05400,5.0,5.606605,6.989244,2.024949,10.00000,2.044823,5.264754,7.064905,4.560325,4.930271,5.518500
3,2016,ARG,Argentina,4.84,160.0,4.0,5.335294,6.048930,6.0,4.0,5.346056,3.689690,2.930563,4.255995,7.500000,3.333333,3.632872,6.857195,3.385786,4.133832,0.769231,3.904030,5.233264,5.224079,2.00000,10.0,5.614336,6.421600,4.811105,0.00000,4.697482,3.982547,5.419820,5.151405,5.535831,5.369019
4,2016,ARM,Armenia,7.57,29.0,1.0,7.264706,7.748532,8.0,5.0,7.003310,3.867379,4.197569,5.664317,5.833333,5.000000,5.196693,9.801963,5.714028,7.007654,1.000000,5.809215,9.078264,9.263496,9.74600,10.0,9.521940,8.547556,7.194410,10.00000,6.830998,8.143241,9.102046,6.234630,6.797530,7.378069


In [8]:
# Life Expectancy

df_le.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [9]:
#Renaming columns to make year and county column name consistant b/w df's

df_ef['Year']      = df_ef['year']
df_ef['Country']   = df_ef['countries']
df_le['Life expectancy'] = df_le['Life expectancy ']

In [10]:
# Merging dataframe

merged_df = df_ef.merge(df_le, on=['Year', 'Country'])

In [11]:
# Interested in Life expectancy and ECONOMIC FREEDOM

interested_cols = ['Year', 'ISO_code','Country','ECONOMIC FREEDOM', 'Adult Mortality', 'infant deaths', 'Life expectancy', 'Population', 'Status']
clean_df = merged_df[interested_cols]

In [12]:
# Check if : (all the merged rows)  and (where count is ==16 rows) are same

print(clean_df.groupby(by='Country').count()['Year'].shape[0])
print(clean_df.groupby(by='Country').count() [ clean_df.groupby(by='Country').count()['Year']==16].shape[0])

133
133


In [13]:
# Checking Unique Years in clean_df

clean_df.Year.unique()

array([2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005,
       2004, 2003, 2002, 2001, 2000])

In [14]:
# Plotting time series 

clean_df = clean_df.dropna()
data_slider = []
steps = []

# Popluate data_slider
for year in clean_df.Year.unique():
    if year !=2000:
      df1   = clean_df[clean_df['Year']== year]
      
      temp_Df = df1.copy()
      temp_Df.dropna()

      # Column z: Higher value means more correlation 
      temp_Df['z'] = df1['ECONOMIC FREEDOM'].corr(df1['Life expectancy'])


      data_one_year = dict(
                          type='choropleth',
                          locations = temp_Df['ISO_code'],
                          z=temp_Df['z'],
                          locationmode='ISO-3',
                          colorscale = 'Sunsetdark'
                          )

      data_slider.append(data_one_year) 

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                #Label to be displayed for each step (year)
                label='Year {}'.format(i + 2001)) 
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  
layout = dict(geo=dict(scope='world'), sliders=sliders)
fig = dict(data=data_slider, layout=layout) 

plotly.offline.iplot(fig)

In [15]:

!pip install bubbly
from bubbly.bubbly import bubbleplot

  Created wheel for bubbly: filename=bubbly-1.0.2-cp36-none-any.whl size=5429 sha256=a5b89a310f974bd805909e579f8eb7a29248b29b3b03f503330c5fde9f34d113
  Stored in directory: /root/.cache/pip/wheels/cf/3c/75/8c09dbf7cd6b320b69a6bc679fd111abd1139866c722b1476b
Successfully built bubbly


In [16]:
clean_df = clean_df.sort_values(by=['Year'])

In [17]:
figure = bubbleplot(dataset=clean_df, x_column='ECONOMIC FREEDOM', y_column='Life expectancy', 
                    bubble_column='Country', time_column='Year', size_column='Population', color_column='Status', 
                    x_title="Economic Freddom", y_title="Life Expectancy",
                    x_logscale=True, scale_bubble=3, height=650)
plotly.offline.iplot(figure, config={'scrollzoom': True})

/usr/local/lib/python3.6/dist-packages/plotly/offline/offline.py:159: UserWarning:


Unrecognized config options supplied: ['scrollzoom']



In [18]:
conti['Country'] = conti['name']
conti = conti[['Country', 'region']]
conti

,Country,region
0,Afghanistan,Asia
1,Åland Islands,Europe
2,Albania,Europe
3,Algeria,Africa
4,American Samoa,Oceania
...,...,...
244,Wallis and Futuna,Oceania
245,Western Sahara,Africa
246,Yemen,Asia
247,Zambia,Africa


In [19]:
clean_df

,Year,ISO_code,Country,ECONOMIC FREEDOM,Adult Mortality,infant deaths,Life expectancy,Population,Status
2127,2000,ZWE,Zimbabwe,4.51,665.0,24,46.0,12222251.0,Developing
2032,2000,SLV,El Salvador,7.72,218.0,4,69.0,5867626.0,Developing
2033,2000,EST,Estonia,7.60,218.0,0,78.0,1396985.0,Developing
2035,2000,FJI,Fiji,6.65,221.0,0,67.7,811223.0,Developing
2036,2000,FIN,Finland,7.98,15.0,0,77.5,517629.0,Developing
...,...,...,...,...,...,...,...,...,...
94,2015,NGA,Nigeria,6.39,344.0,483,54.5,181181744.0,Developing
95,2015,NOR,Norway,7.65,59.0,0,81.8,518867.0,Developed
97,2015,PAK,Pakistan,5.92,161.0,352,66.4,18938513.0,Developing
83,2015,MNG,Mongolia,7.38,222.0,1,68.8,2976877.0,Developing


In [20]:
df_with_continenets = clean_df.merge(conti, on=['Country'])
df_with_continenets

,Year,ISO_code,Country,ECONOMIC FREEDOM,Adult Mortality,infant deaths,Life expectancy,Population,Status,region
0,2000,ZWE,Zimbabwe,4.51,665.0,24,46.0,12222251.0,Developing,Africa
1,2001,ZWE,Zimbabwe,3.87,686.0,25,45.3,12366165.0,Developing,Africa
2,2002,ZWE,Zimbabwe,3.74,73.0,25,44.8,125525.0,Developing,Africa
3,2003,ZWE,Zimbabwe,3.82,715.0,26,44.5,12633897.0,Developing,Africa
4,2004,ZWE,Zimbabwe,3.25,723.0,27,44.3,12777511.0,Developing,Africa
...,...,...,...,...,...,...,...,...,...,...
1674,2013,SYC,Seychelles,7.24,172.0,0,72.9,89949.0,Developing,Africa
1675,2014,SYC,Seychelles,7.46,17.0,0,73.0,91359.0,Developing,Africa
1676,2015,SYC,Seychelles,7.33,168.0,0,73.2,93419.0,Developing,Africa
1677,2014,LBR,Liberia,7.18,329.0,8,58.1,439737.0,Developing,Africa


In [21]:
df_with_continenets = df_with_continenets.groupby(['Year', 'region']).mean().reset_index()
df_with_continenets

,Year,region,ECONOMIC FREEDOM,Adult Mortality,infant deaths,Life expectancy,Population
0,2000,Africa,5.863077,299.730769,56.153846,54.326923,7.372275e+06
1,2000,Americas,6.966500,150.150000,14.400000,72.445000,1.347437e+07
2,2000,Asia,6.221875,148.125000,210.062500,69.925000,2.470027e+07
3,2000,Europe,7.257586,94.241379,1.068966,76.289655,5.858037e+06
4,2000,Oceania,7.420000,149.500000,0.500000,73.600000,4.151880e+05
...,...,...,...,...,...,...,...
75,2015,Africa,6.241429,229.285714,42.314286,62.691429,1.260439e+07
76,2015,Americas,6.951429,131.238095,6.857143,74.923810,1.038025e+07
77,2015,Asia,6.929615,138.076923,69.384615,72.923077,1.811709e+07
78,2015,Europe,7.430968,86.225806,0.516129,79.787097,9.095802e+06


In [22]:
figure = bubbleplot(dataset=df_with_continenets, x_column='ECONOMIC FREEDOM', y_column='Life expectancy', 
                    bubble_column='region', time_column='Year', size_column='Population', color_column='region', 
                    x_title="Economic Freddom", y_title="Life Expectancy", 
                    x_logscale=True, scale_bubble=3, height=650)
plotly.offline.iplot(figure, config={'scrollzoom': True})

/usr/local/lib/python3.6/dist-packages/plotly/offline/offline.py:159: UserWarning:


Unrecognized config options supplied: ['scrollzoom']

